<a href="https://colab.research.google.com/github/ymoslem/Adaptive-MT-LLM/blob/main/data%20/fuzzy_matches/SimilarTranslations-AdaptiveMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download new dataset

In [1]:
import requests

srclang = "en"
tgtlang = "es"

opus_url = "https://opus.nlpl.eu/opusapi/?source="+srclang+"&target="+tgtlang+"&preprocessing=moses&version=latest"
response = requests.get(opus_url)
response_json = response.json()
corpora = response_json["corpora"]

for corpus in corpora:
  if corpus["corpus"] == "tico-19":
    url = corpus["url"]
    print(corpus)

{'alignment_pairs': 3071, 'corpus': 'tico-19', 'documents': '', 'id': 482305, 'latest': 'True', 'preprocessing': 'moses', 'size': 335, 'source': 'en', 'source_tokens': 70375, 'target': 'es', 'target_tokens': 85484, 'url': 'https://object.pouta.csc.fi/OPUS-tico-19/v2020-10-28/moses/en-es.txt.zip', 'version': 'v2020-10-28'}


In [7]:
source_file = f"tico-19.{srclang}-{tgtlang}.{srclang}"
target_file = f"tico-19.{srclang}-{tgtlang}.{tgtlang}"

In [16]:
zip_file = f"{srclang}-{tgtlang}.txt.zip"
xml_file = f"tico-19.{srclang}-{tgtlang}.xml"

!wget $url
!unzip $zip_file
!rm README LICENSE $xml_file

--2023-12-14 15:31:12--  https://object.pouta.csc.fi/OPUS-tico-19/v2020-10-28/moses/en-es.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343223 (335K) [application/zip]
Saving to: ‘en-es.txt.zip’

en-es.txt.zip       100%[===================>] 335.18K   412KB/s    in 0.8s    

2023-12-14 15:31:14 (412 KB/s) - ‘en-es.txt.zip’ saved [343223/343223]

Archive:  en-es.txt.zip
  inflating: README                  
  inflating: LICENSE                 
  inflating: tico-19.en-es.en        
  inflating: tico-19.en-es.es        
  inflating: tico-19.en-es.xml       


In [17]:
with open(source_file) as source_file, open(target_file) as target_file:
  source_sents = [sent.strip() for sent in source_file.readlines()]
  target_sents = [sent.strip() for sent in target_file.readlines()]

  sent_pairs = [sent_pair for sent_pair in zip(source_sents, target_sents)]
  print("All sentences:", len(sent_pairs))
  sent_pairs = set(sent_pairs)
  print("Unique sentences:", len(sent_pairs))

  source_sentences = [source for source, target in sent_pairs]
  target_sentences = [target for source, target in sent_pairs]

  print(source_sentences[0])
  print(target_sentences[0])

All sentences: 3071
Unique sentences: 3070
In China, there have been incidents of foreigners being refused rooms by hotel staff, and foreign residents, particularly black people, being kicked out of their apartments on short notice by their landlords.
En China, ha habido incidentes en los que el personal de algunos hoteles se ha negado a darles una habitación a extranjeros y en los que algunos propietarios de apartamentos han echado de los apartamentos con poco tiempo de aviso a sus inquilinos extranjeros, en especial a inquilinos negros.


In [18]:
output_source_file = f"tico-19-{srclang}{tgtlang}-dedup.{srclang}"
output_target_file = f"tico-19-{srclang}{tgtlang}-dedup.{tgtlang}"

# Save sources
with open(output_source_file, "w+") as source:
  for line in source_sentences:
    source.write(line.strip() + "\n")

# Save target references
with open(output_target_file, "w+") as target:
  for line in target_sentences:
    target.write(line.strip() + "\n")

## Load datasets

In [19]:
lang = "es" #@param ["ar", "es", "fr", "rw", "zh"]
source_file_name = "tico-19-enes-dedup.en" #@param ["tico-19-enar-dedup.en", "tico-19-enes-dedup.en", "tico-19-enfr-dedup.en", "tico-19-enrw-dedup.en", "tico-19-enzh-dedup.en"]
target_file_name = "tico-19-enes-dedup.es" #@param ["tico-19-enar-dedup.ar", "tico-19-enes-dedup.es", "tico-19-enfr-dedup.fr", "tico-19-enrw-dedup.rw", "tico-19-enzh-dedup.zh"]


# Load source
with open(source_file_name, "r") as source, open(target_file_name, "r") as target:
  source_sentences = [sent.strip() for sent in source.readlines()]
  target_sentences = [sent.strip() for sent in target.readlines()]

print(len(source_sentences), len(target_sentences))
print(0, source_sentences[0], target_sentences[0], sep="\n")
print(-1, source_sentences[-1], target_sentences[-1], sep="\n")

3070 3070
0
In China, there have been incidents of foreigners being refused rooms by hotel staff, and foreign residents, particularly black people, being kicked out of their apartments on short notice by their landlords.
En China, ha habido incidentes en los que el personal de algunos hoteles se ha negado a darles una habitación a extranjeros y en los que algunos propietarios de apartamentos han echado de los apartamentos con poco tiempo de aviso a sus inquilinos extranjeros, en especial a inquilinos negros.
-1
Coronaviruses (CoVs) belong to the family Coronaviridae, which comprises a group of enveloped, positive-sensed, single-stranded RNA viruses.
Los coronavirus pertenecen a la familia Coronaviridae, que comprende un grupo de virus de ARN monocatenario positivo con envoltura.


## Find similar sentences

In [20]:
!pip3 install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00


In [21]:
print(len(source_sentences))

3070


In [22]:

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

# For 1-shot k=1; for n-shot k=n
k = 1 #@param {type:"slider", min:1, max:20, step:1}
paraphrases = util.paraphrase_mining(model, source_sentences, corpus_chunk_size=len(source_sentences), top_k=k)

print("Done! K =", k)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Done! K = 1


In [23]:
len(paraphrases)

2473

In [24]:
# Sanity check

n = 55

paraphrases[n]
i = paraphrases[n][1]
j = paraphrases[n][2]

print(source_sentences[j],
      target_sentences[j],
      "",
      source_sentences[i],
      target_sentences[i],
      sep="\n")

The virus is highly homologous to the coronavirus (CoV) that caused an outbreak of severe acute respiratory syndrome (SARS) in 2003; thus, it was named SARS-CoV-2 by the World Health Organization (WHO) on February 11, 2020, and the associated disease was named CoV Disease-19 (COVID-19).
El virus tiene casi el mismo nombre que el coronavirus (CoV) que causó un brote de síndrome respiratorio agudo severo (SARS) en 2003; por eso, el 11 de febrero de 2020, la Organización Mundial de la Salud (OMS) lo llamó SARS-CoV-2, y la enfermedad asociada se denominó enfermedad por coronavirus 2019 (COVID-19).

Since, the illness resulting from SARS-CoV-2 infection has been named coronavirus disease (COVID-19).
Desde entonces, la enfermedad provocada por la infección con SARS-CoV-2 se ha denominado enfermedad por coronavirus (COVID-19).


In [26]:
for paraphrase in paraphrases[:10]:
  score, i, j = paraphrase
  print("• {} {}\n• {} {}\nScore: {:.4f}\n".format(i, source_sentences[i], j, source_sentences[j], score))

• 848 When there is minimal to moderate community transmission, social distancing strategies can be implemented such as cancelling field trips, assemblies, and other large gatherings such as physical education or choir classes or meals in a cafeteria, increasing the space between desks, staggering arrival and dismissal times, limiting nonessential visitors, and using a separate health office location for children with flu-like symptoms.
• 1779 When there is minimal to moderate community transmission, social distancing strategies can be implemented such as canceling field trips, assemblies, and other large gatherings such as physical education or choir classes or meals in a cafeteria, increasing the space between desks, staggering arrival and dismissal times, limiting nonessential visitors, and using a separate health office location for children with flu-like symptoms.
Score: 0.9997

• 39 Some medical procedures such as intubation and cardiopulmonary resuscitation (CPR) may cause respi

## Create JSON and save

In [28]:
all_paraphrases = []

for paraphrase in paraphrases:
  score, i, j = paraphrase

  paraphrase_dict = {}
  paraphrase_dict["score"] = score
  paraphrase_dict["origin_src"] = source_sentences[i]
  paraphrase_dict["origin_tgt"] = target_sentences[i]
  paraphrase_dict["match_src"] = source_sentences[j]
  paraphrase_dict["match_tgt"] = target_sentences[j]

  all_paraphrases.append(paraphrase_dict)

In [29]:
all_paraphrases[500]

{'score': 0.7772939801216125,
 'origin_src': 'There is no available vaccine, but various agencies are actively developing vaccine candidates.',
 'origin_tgt': 'No existe una vacuna, pero varias agencias están desarrollando activamente posibles vacunas.',
 'match_src': 'There is no COVID-19 vaccine, though many organizations are working to develop one.',
 'match_tgt': 'No hay una vacuna contra la COVID-19, aunque muchas organizaciones están trabajando para desarrollar una.'}

In [30]:
# Save the JSON file
import json

# Change output file name
output_file_name ="similar-"+ str(k) +"-tico." + lang +".json"

with open(output_file_name, "w+") as file_path:
  json.dump(all_paraphrases, file_path)
  print("Done writing JSON data into the file:", output_file_name)

Done writing JSON data into the file: similar-1-tico.es.json


In [31]:
# Load the JSON file
import json

with open(output_file_name, 'rb') as file_path:
  loaded_paraphrases = json.load(file_path)

loaded_paraphrases[500]

{'score': 0.7772939801216125,
 'origin_src': 'There is no available vaccine, but various agencies are actively developing vaccine candidates.',
 'origin_tgt': 'No existe una vacuna, pero varias agencias están desarrollando activamente posibles vacunas.',
 'match_src': 'There is no COVID-19 vaccine, though many organizations are working to develop one.',
 'match_tgt': 'No hay una vacuna contra la COVID-19, aunque muchas organizaciones están trabajando para desarrollar una.'}

In [32]:
len(loaded_paraphrases)

2473